First attempt seems to be correct, but non-performant.

In [ ]:
# Given an integer k and a set S of distinct integers, print the size of a maximal subset
# S' of S such that the sum of any two numbers s1, s2 in S' in not evenly divisible by k.

# the integers actually come as an array with possible duplicates, so first we convert to set 

import itertools

arr = [19, 10, 12, 10, 24, 25, 22]
S = set(arr)
k = 4

S = set([1, 7, 2, 4])
k = 3

def nonDivisibleSubset(k, S):
    for i in range(len(S), -1, -1):
        for combo in itertools.combinations(S, i):
            if all([sum(pair) % k != 0 \
                    for pair in itertools.combinations(combo, 2)]):
                return i
    return 0

print(nonDivisibleSubset(k, S))

To solve this problem more efficiently, we can use the following Theorem.

__Theorem__.  The sum $a+b$ of integers $a$ and $b$ is divisible by $k\in\mathbb{Z}$ if and only if:

$$\left(a \text{ mod } k \right) + \left(b \text{ mod } k \right) = k $$

or

$$\left(a \text{ mod } k \right) + \left(b \text{ mod } k \right) = 0. $$


__Proof__.  It's been a while, so my proof writing is a bit rusty.  But I'll give it go.  

Suppose that $k\mid (a+b)$.  By the division algorithm,

$$a = k\cdot q_1 + r_1$$

and

$$b = k\cdot q_2 + r_2$$

for integers $q_1, q_2, r_1$ and $r_2$.  Combining these three equations, we have 

\begin{align}
a + b &= (k\cdot q_1 + r_1) + (k\cdot q_2 + r_2)\\
         &= k(q_1+q_2) + (r_1 + r_2).
\end{align}

Now, the right side of this equation will be divisible by $k$ only if both of its summands are divisible by $k$.  That is, if $r_1+r_2$ is a multiple of $k$, or is zero.  Since both $r_1$ and $r_2$ are at most $k-1$, their sum must be less than $2k$, implying that said sum must be divisible by $k$ only if it equals $k$ or is zero.  This proves the first part of the theorem.  

The reverse implication is established by a straightforward computation.  Using the same notation as in the first part, 

$$r_1 + r_2 = 0$$
$\Rightarrow$
$$a + b = k(q_1 + q_2) + (r_1 + r_2) = k(q_1 + q_2)$$
$\Rightarrow$
$$k\mid a+b,$$

whereas

$$r_1 + r_2 = k$$
$\Rightarrow$
$$a + b = k(q_1 + q_2) + (r_1 + r_2) = k(q_1 + q_2) + k = k(1+q_1+q_2)$$
$\Rightarrow$
$$k\mid a+b.\Box$$


For some reason \LaTex `$\Box$` is showing up as a rhombus.  And `\LaTex` is not displaying prettily.  There is an [issue](https://github.com/jupyter/nbconvert/issues/719) for the latter.  I'm not sure about the former.  Next time I suppose I should use the theorem environment.


In [55]:
# Here is a small example

a, b = 3, 7
ks = [1, 2, 3, 5, 7, 10, 11, 12] 
to_be = ["isn't", "is"]

for k in ks:
    print("{} + {} {} divisible by {}:".format(a, b, to_be[(a+b)%k == 0], k))
    print("  Computing modulo of sum:")
    print("    Since ({}+{}) % {} = {} % {} = {} ".format(a, b, k, a+b, k, (a+b)%k))
    print("  Computing sum of modulos:")
    print("    Since ({} % {}) + ({} % {}) = {} + {} = {}".format(a, k, b, k, a%k, b%k, a%k + b%k))
    #print("  Computed:")
    #print("    {}%{} + {}%{} = {}")
    print()


3 + 7 is divisible by 1:
  Computing modulo of sum:
    Since (3+7) % 1 = 10 % 1 = 0 
  Computing sum of modulos:
    Since (3 % 1) + (7 % 1) = 0 + 0 = 0

3 + 7 is divisible by 2:
  Computing modulo of sum:
    Since (3+7) % 2 = 10 % 2 = 0 
  Computing sum of modulos:
    Since (3 % 2) + (7 % 2) = 1 + 1 = 2

3 + 7 isn't divisible by 3:
  Computing modulo of sum:
    Since (3+7) % 3 = 10 % 3 = 1 
  Computing sum of modulos:
    Since (3 % 3) + (7 % 3) = 0 + 1 = 1

3 + 7 is divisible by 5:
  Computing modulo of sum:
    Since (3+7) % 5 = 10 % 5 = 0 
  Computing sum of modulos:
    Since (3 % 5) + (7 % 5) = 3 + 2 = 5

3 + 7 isn't divisible by 7:
  Computing modulo of sum:
    Since (3+7) % 7 = 10 % 7 = 3 
  Computing sum of modulos:
    Since (3 % 7) + (7 % 7) = 3 + 0 = 3

3 + 7 is divisible by 10:
  Computing modulo of sum:
    Since (3+7) % 10 = 10 % 10 = 0 
  Computing sum of modulos:
    Since (3 % 10) + (7 % 10) = 3 + 7 = 10

3 + 7 isn't divisible by 11:
  Computing modulo of sum:
  

We will use our theorem to solve the problem in the following way.  Recall that we need to find the maximal subset such that all pairs are _not_ divisible by $k$.  Thus, 

1.  we can include no pairs whose remainders sum to $k$, and
2.  we can include no more than a single multiple of $k$.

To find the maximal such subset, we can choose, for each pair of remainders summing to $k$ (ie, $1+(k-1)$, $2+(k-2)$ and so on) choose the remainder with the most corresponding elements.  Once we have checked all such pairs, we throw in one multiple of $k$ (if there is one) and call it a day.  

In [136]:
# Second attempt, which performs somewhat better, but still fails half of the test cases due to time out.

import itertools
import collections
import math

def nonDivisibleSubset(k, S):
    S_count = 0
    remainders = {r:[s_ for s_ in S if s_%k == r] for s in S for r in range(k)}
    
    for r1, r2 in zip(range(1, math.floor(k/2) + 1), range(k-1, math.ceil(k/2) - 1, -1)):
        if r1 == r2:
            S_count += len(remainders[r1])>0 or len(remainders[r2])>0
        elif len(remainders[r1]) >= len(remainders[r2]):
            S_count += len(remainders[r1])
        else:
            S_count += len(remainders[r2]) 
    if len(remainders[0]) > 0:
        S_count += 1
        
    return S_count

print(nonDivisibleSubset(k, S))

3


Well, we still have perfomance issues.  Looking into the time complexity of our function (ignoring the `remainders` dictionary for now):

```
For loop:  O(k/2)
    if:        O(1)
    elif:      O(1)
    else:      O(1)
if:        O(1)
```

for a grand total of $O(k)$, which certainly seems pretty good.  Although I might be mistaken in my analysis.  I suspect that the complexity culprit is in my neat little dictionary comprehension:

```python
{r:[s_ for s_ in S if s_%k == r] for s in S for r in range(k)}
```

This is actually a relic from an intermediate version of my function, which was returning an instance of a maximal subset, not just its size.

In [137]:
# third attempt

def nonDivisibleSubset(k, S):
    S_count = 0
    remainders = {r:0 for r in range(k)}
    for s in S:
        remainders[s%k] += 1
    for r1, r2 in zip(range(1, math.floor(k/2) + 1), range(k-1, math.ceil(k/2) - 1, -1)):
        if r1 == r2:
            S_count += remainders[r1]>0 or remainders[r2]>0
            
        elif remainders[r1] >= remainders[r2]:
            S_count += remainders[r1]
        else:
            S_count += remainders[r2] 
    if remainders[0] > 0:
        S_count += 1
    return S_count

And that did the trick.  Hooray!

In [138]:
sets = [set([19, 10, 12, 10, 24, 25, 22]),
        set([1, 7, 2, 4]),
        set([278, 576, 496, 727, 410, 124, 338, 149, 209, 702, 282, 718, 771, 575, 436]),
        set([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
         ]

ks = [4, 3, 7, 4]

expected = [3, 3, 11, 5]

def test(sets, ks, expected):
    test = 0
    failures = []
    for S, k, (i, expected) in zip(sets, ks, enumerate(expected)):
        test += 1
        print("Test {}".format(i))
        print("  Expected:", expected)
        print("  Computed:", nonDivisibleSubset(k, S))
        if expected != nonDivisibleSubset(k,S):
            print("Test {} failed!".format(i))
            failures.append(i)
        print("====================")
    print("Failed the following tests: {}".format(failures))
           
test(sets, ks, expected)


Test 0
  Expected: 3
{0: 2, 1: 1, 2: 2, 3: 1}
  Computed: 3
{0: 2, 1: 1, 2: 2, 3: 1}
Test 1
  Expected: 3
{0: 0, 1: 3, 2: 1}
  Computed: 3
{0: 0, 1: 3, 2: 1}
Test 2
  Expected: 11
{0: 0, 1: 2, 2: 6, 3: 0, 4: 2, 5: 2, 6: 3}
  Computed: 11
{0: 0, 1: 2, 2: 6, 3: 0, 4: 2, 5: 2, 6: 3}
Test 3
  Expected: 5
{0: 2, 1: 3, 2: 3, 3: 2}
  Computed: 5
{0: 2, 1: 3, 2: 3, 3: 2}
Failed the following tests: []


In [61]:
import collections
collections.Counter?

Init signature: collections.Counter(*args, **kwds)
Docstring:     
Dict subclass for counting hashable items.  Sometimes called a bag
or multiset.  Elements are stored as dictionary keys and their counts
are stored as dictionary values.

>>> c = Counter('abcdeabcdabcaba')  # count elements from a string

>>> c.most_common(3)                # three most common elements
[('a', 5), ('b', 4), ('c', 3)]
>>> sorted(c)                       # list all unique elements
['a', 'b', 'c', 'd', 'e']
>>> ''.join(sorted(c.elements()))   # list elements with repetitions
'aaaaabbbbcccdde'
>>> sum(c.values())                 # total of all counts
15

>>> c['a']                          # count of letter 'a'
5
>>> for elem in 'shazam':           # update counts from an iterable
...     c[elem] += 1                # by adding 1 to each element's count
>>> c['a']                          # now there are seven 'a'
7
>>> del c['b']                      # remove all 'b'
>>> c['b']                          # no